In [33]:
%%capture
%run 01_hash.ipynb
%run 02_transaction.ipynb
%run 05_block.ipynb

In [34]:
import subprocess as sub

## TODO:

- differentiate between storage/local variables

Only store storage variables in the global storage.

In [23]:
c = """
storage a = 12
storage abc = 12
"""

In [24]:
def storage_vars(code):
    matches = re.findall(r'storage [a-z]*', e)
    return [m.replace('storage', '').strip() for m in matches]
    
storage_vars(c)

['a', 'abc']

## 99_contract

Our goal is to be able to run a small piece of code in a tx, just like Ethereum. This piece of code is also called a smart contract (sc). The sc is like another user, but in addition to the balance it also has storage for variables and the sc itself. 

The smart contract should be able to access the current tx and block information. These will get injected as `_tx` and `_b`.

The smart contract that we are going to run contains the following code:

In [35]:
code = """
def init():
    r = 0
    s = 2
    h = 'hello how are you?'
    l = ['this', 'is', 'a', 'list']
    b = _b['number'] + 100

r += 5
time = _tx['time'] + 'XXXXX'
l.append('XXXXX')
"""

Temporary file name where code for execution will be stored.

In [36]:
F = 'temp.py'

This is injected into every execution to get all the variables used in the contract.

In [37]:
PRINT_VARS = ['local = set(locals())',
              'local_vars = {}',
              'for n in local:',
              '    if not n.startswith(\'_\'):',
              '         if \'str\' in str(type(eval(n))): print(n, f\'\"{eval(n)}\"\', \'str\')',
              '         else: print(n, eval(n), \'int\')']

In [51]:
PRINT_VARS = """
\n
import json
_types = [str, float, int, list]                                                                         
out = {}                                                                                                 
keys = list(locals().keys())                                                                             
for k in keys:                                                                                           
    if not k.startswith('_'):                                                                            
        v = locals()[k]                                                                                  
        for t in _types:                                                                                 
            if isinstance(v, t): out[k] = [v, str(t)]                                                    
print(json.dumps(out)) 
"""
PRINT_VARS = PRINT_VARS.rstrip()

All vars used will be stored in the `STORAGE` dict.

In [39]:
STORAGE = {}

Inject transaction `tx` and `block` into the execution. These can be accessed in the sc.

In [40]:
def inject(tx, code, block=None): 
    tx = tx.json().replace('true', 'True').replace('false', 'False')
    b = block.json().replace('true', 'True').replace('false', 'False')
    return '_tx = '+tx+'\n'+'_b = '+b+'\n'+code

In [41]:
def save(code):
    with open(F, 'w+') as f: f.write(code)

In [42]:
def run():
    r = sub.check_output(f'python3 {F}', shell=True)
    return json.loads(r)

Stores all variables in `STORAGE`.

In [43]:
def store(r):
    for k,v in r.items(): STORAGE[k] = [v[0],v[1]]

The deployment will run all the code in `init()`. The rest will run in `execution`.

In [44]:
def deployable(code):
    init = []
    lines = code.split('\n')
    for i,l in enumerate(lines):
        if 'init' in l: 
            for j in range(i+1, len(lines)):
                if lines[j][:1] == ' ': init.append(lines[j].strip())
                else                  : break
    init.append(PRINT_VARS)
    return '\n'.join(l for l in init)

In [45]:
def deploy(code, tx):
    dc = deployable(code)
    ic = inject(tx, dc, b)
    save(ic)
    r  = run()
    store(r)

Execution will run everything in the sc except what is inside `init()`.

In [46]:
def executable(code):
    exe = []
    for k,v in STORAGE.items():
        if 'str' in v[1]: exe.append(f'{k} = "{v[0]}"')
        else            : exe.append(f'{k} = {v[0]}')
    lines = code.split('\n')
    in_init = False
    for l in lines:
        if 'init' in l: in_init = True; continue
        if in_init:
            if l[:1] == ' ': continue
            else           : in_init = False
        else: exe.append(l)
    exe.append(PRINT_VARS)
    return '\n'.join(l for l in exe)

In [47]:
def exe(code, tx):
    ic  = inject(tx, code, b)
    exe = executable(ic)
    save(exe)
    r  = run()
    store(r)

## Smart Contract

Stores all the necesary information for deploying and running smart contract `code`. 

The `tx` and `b` should be injected when the contract is run on the blockchain. This way the contract has always access to its tx and current block.

In [53]:
class SmartContract(Hashable):
    def __init__(self, code):
        self.code     = code
        self.deployed = False
        self.time     = time.ctime()
        self.runs     = 0
        
    def run(self, code, tx, b):
        ic = inject(tx, code, b)
        save(ic)
        r  = run()
        store(r)
        
    def deploy(self, tx, b): 
        dc = deployable(self.code)
        self.run(dc, tx, b)
        self.deployed = True
        
    def exe(self, tx, b): 
        assert self.deployed, 'Deploy Contract first'
        ic  = executable(self.code)
        self.run(ic, tx, b)
        self.runs += 1
    
sc = SmartContract(code)
sc.deploy(tx1, b)
STORAGE

{'r': [0, "<class 'int'>"],
 's': [2, "<class 'int'>"],
 'h': ['hello how are you?', "<class 'str'>"],
 'l': [['this', 'is', 'a', 'list'], "<class 'list'>"],
 'b': [102, "<class 'int'>"],
 'time': ['Mon Apr 12 20:21:52 2021XXXXX', "<class 'str'>"]}

In [54]:
sc.exe(tx1, b)
STORAGE

{'r': [5, "<class 'int'>"],
 's': [2, "<class 'int'>"],
 'h': ['hello how are you?', "<class 'str'>"],
 'l': [['this', 'is', 'a', 'list', 'XXXXX'], "<class 'list'>"],
 'b': [102, "<class 'int'>"],
 'time': ['Mon Apr 12 20:21:52 2021XXXXX', "<class 'str'>"]}